In [9]:
import yaml

def load_settings(file_path):
    with open(file_path, 'r') as stream:
        try:
            settings = yaml.safe_load(stream)
            return settings
        except yaml.YAMLError as exc:
            print(f"Error loading YAML file: {exc}")

file_path = '../../../settings.yml'
settings = load_settings(file_path)

In [10]:
import csv
import string
import random
AIRCRAFT_COUNT = int(settings['data_generation']['aircrafts'])
CARRIER_COUNT = int(settings['data_generation']['carriers'])

In [11]:
x = random.randint(1, 2**7 -1 )
xb = "{0:08b}".format(x)[1:]
m = list(map(lambda x: True if x == '1' else False, xb))
f = sum(list(map(int, xb)))

print(x, f, m, xb)

46 4 [False, True, False, True, True, True, False] 0101110


In [12]:
## Generate carreir.csv

with open('carrier.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile, delimiter=',')
    writer.writerow(['code', 'desc'])
    letters = string.ascii_uppercase

    for i in range(1, CARRIER_COUNT + 1):
        code = ''.join(random.choice(letters) for _ in range(2))   # 2 letter code
        writer.writerow([code, 'carrier_' + code])
    

In [13]:
## Generate Aircrafts and their seat distribution

passenger_aircraft_models = [
    "Boeing 737",
    "Airbus A320",
    "Boeing 747",
    "Airbus A380",
    "Boeing 777",
    "Airbus A330",
    "Boeing 787 Dreamliner",
    "Embraer E190",
    "Bombardier CRJ700",
    "McDonnell Douglas MD-80",
    "Cessna 172",
    "Gulfstream G650",
    "Beechcraft King Air",
    "ATR 72",
    "Bombardier Global 6000",
    "Piper PA-28 Cherokee"
]

owner = {
    "Air India" : "AI",
    "IndiGo" : "6E",
    "SpiceJet" : "SG",
    "GoAir" : "G8",
    "Vistara" : "UK",
    "AirAsia India" : "I5",
    "Air India Express" : "IX",
    "AirAsia" : "AK",
    "AirAsia X" : "D7",
    "Malindo Air" : "OD",
}

capacity = [72, 100, 150, 270, 360, 300, 400]
passenger_aircraft_models = {key: random.choice(capacity) for key in passenger_aircraft_models}

classes = {
    "F" : ["F", "P"],
    "B" : ["C", "J", "Z"],
    "P" : ["Q", "R" , "S", "T", "H", "M"],
    "E" : ['Y', 'A', 'B', 'D', 'G', 'I', 'E', 'K', 'L', 'N', 'O', 'U', 'V', 'W', 'X']
}


with open('aircrafts.csv', 'w') as csvfile:
    with open('seat-distribution.csv', 'w') as seat_distribution_csvfile:
        # model, name, registration, owner_code, owner_name, total_capacity, total_inventory
        writer = csv.writer(csvfile, delimiter=',')
        writer.writerow(['id','model', 'name', 'registration', 'owner_code', 'owner_name', 'total_capacity', 'total_inventory'])

        writer_sd = csv.writer(seat_distribution_csvfile, delimiter=',')
        writer_sd.writerow(['aircraft_id', 'class', 'seat_count'])


        models = list(passenger_aircraft_models.keys())

        for i in range(AIRCRAFT_COUNT):
            model = random.choice(models)
            name = ""
            registration = "VT-" + str(random.randint(1000, 9999))
            owner_name = random.choice(list(owner.keys()))
            owner_code = owner[owner_name] + str(random.randint(100, 999))
            total_capacity = passenger_aircraft_models[model]

            t1, t2  = total_capacity, 0
            p = 10
            for cabin_name in classes.keys():
                t = random.randint(0, 100)
                total_allocation = int(total_capacity * p / 100)
                if p == 40: total_allocation = total_capacity
                total_capacity -= total_allocation


                p  += 10
                if t < 10 and p < 40:
                    continue

                for seat_class in classes[cabin_name]:
                    seat_count = max(int(total_allocation / len(classes[cabin_name])) + random.randint(-3, 3), 0)
                    # total_allocation -= seat_count
                    t2 += seat_count
                    writer_sd.writerow([i+1, seat_class, seat_count])
                
            t2_i = t2 + int(t2 * random.randint(10, 20) / 100)
            writer.writerow([i+1, model, name, registration, owner_code, owner_name, t2, t2_i])

    seat_distribution_csvfile.close()
csvfile.close() 